In [7]:
from datetime import date

In [8]:
now = date.today()
now

datetime.date(2022, 10, 18)

In [9]:
customer_id = '0140'

In [10]:
query = f'''http://192.168.1.104/ReportServer/Pages/ReportViewer.aspx\
?%2fDynamicsAX%2fCustPriceList.PrecisionDesign&CustPriceList_CustAccount={customer_id}\
&CustPriceList_PerDate={now}&CustPriceList_CurrencyCode=usd\
&CustPriceList_DataArea=dka&rs:Format=Excel'''

In [11]:
query

'http://192.168.1.104/ReportServer/Pages/ReportViewer.aspx?%2fDynamicsAX%2fCustPriceList.PrecisionDesign&CustPriceList_CustAccount=0140&CustPriceList_PerDate=2022-10-18&CustPriceList_CurrencyCode=usd&CustPriceList_DataArea=dka&rs:Format=Excel'

In [13]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [5]:
import requests
from requests_ntlm import HttpNtlmAuth

In [2]:
username = os.getenv('USERNAME_REPORT_SERVER')
password = os.getenv('PASSWORD_REPORT_SERVER')

In [14]:
username, password

('report_user', 'YandexNavsegda1212')

In [19]:
request = requests.get(query,auth=HttpNtlmAuth(username, password))

In [20]:
request.status_code

500

In [ ]:
if request.status_code == 200:
    with open('price_list.xlsx', 'wb') as out:
        for bits in request.iter_content():
            out.write(bits)

In [ ]:
import pandas as pd
#from StyleFrame import StyleFrame, Styler, utils

In [ ]:
df = pd.read_excel('price_list.xlsx', skiprows=10, usecols="A:C,E,F,I,J,M:S,U", engine='xlrd')
df = df.assign(Order='',Order_Sum='')

In [ ]:
# [opt for opt in dir(request) if not opt.startswith("__")]

In [ ]:
writer = pd.ExcelWriter(f'Price_list-{now}.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Price_list', index=False, startrow=5)

In [ ]:
workbook  = writer.book
worksheet = writer.sheets['Price_list']

In [ ]:
header_format = workbook.add_format({
    'bold': True,
    # 'text_wrap': True,
    'valign': 'top',
    'align': 'center',
    'fg_color': '#41E3A1',
    'border': 1})

In [ ]:
total_format = workbook.add_format({
    'bold': True,
    'border': 1})

In [ ]:
small_header_format = workbook.add_format({
    'bold': True,
    'border': 1,
    'align': 'center',
    'fg_color': '#41E3A1'})

In [ ]:
money_format = workbook.add_format({'num_format': '$# ##0.##',
'border': 1
})

In [ ]:
money_format_ = workbook.add_format({'num_format': '$# ##0.##'
})

In [ ]:
cell_format = workbook.add_format({'border': 1,
'num_format': '# ##0.##'})

In [ ]:
for col_num, value in enumerate(df.columns.values):
    worksheet.write(5, col_num, value, header_format)

In [ ]:
x = (10,13,13,13,16,16,30,13,13,14,12,13,12,14,14,12,19)
for i,x in enumerate(x):
    worksheet.set_column(i,i,x)

In [ ]:
worksheet.set_column(13, 13, 14, money_format_)

In [ ]:
worksheet.write_string('A4', 'Customer:')
worksheet.write_string('B4', f'{customer_id}')
worksheet.write_string('A5', 'Currency:')
worksheet.write_string('B5', 'USD')

In [ ]:
# Tables with reserve totals
worksheet.write_string('N2', 'Total weight', total_format)
worksheet.write_string('N3', 'Total CBM', total_format)
worksheet.write_string('N4', 'Total Cartoons', total_format)
worksheet.write_string('N5', 'Total Sum', total_format)

In [ ]:
# Small table's header
worksheet.write_string('N1', '', small_header_format)
worksheet.write_string('O1', 'Reserved', small_header_format)
worksheet.write_string('P1', 'Order', small_header_format)
worksheet.write_string('Q1', 'Total Res + Ord', small_header_format)

In [ ]:
# add formulas for reserved goods
row_numbers = df.shape[0] + 1
worksheet.write_dynamic_array_formula('O2', f'=SUM($I$7:$I${row_numbers}*$O$7:$O${row_numbers})',cell_format) # Total weight
worksheet.write_dynamic_array_formula('O3', f'=SUM($J$7:$J${row_numbers}*$O$7:$O${row_numbers})',cell_format) # Total CBM
worksheet.write_dynamic_array_formula('O4', f'=SUM($K$7:$K${row_numbers}*$O$7:$O${row_numbers})') # Total Cartoons
worksheet.write_dynamic_array_formula('O5', f'=SUM($N$7:$N${row_numbers}*$O$7:$O${row_numbers})',money_format) # Total SUM

In [ ]:
# add formulas for ordered goods
worksheet.write_dynamic_array_formula('P2', f'=SUM($I$7:$I${row_numbers}*$P$7:$P${row_numbers})',cell_format) # Total weight
worksheet.write_dynamic_array_formula('P3', f'=SUM($J$7:$J${row_numbers}*$P$7:$P${row_numbers})',cell_format) # Total CBM
worksheet.write_dynamic_array_formula('P4', f'=SUM($K$7:$K${row_numbers}*$P$7:$P${row_numbers})') # Total Cartoons
worksheet.write_dynamic_array_formula('P5', f'=SUM($N$7:$N${row_numbers}*$P$7:$P${row_numbers})',money_format) # Total SUM

In [ ]:
worksheet.write_dynamic_array_formula(f'Q7:Q{row_numbers}',    f'=(N7:N{row_numbers}*P7:P{row_numbers})',money_format_)

In [ ]:
# add formulas for ordered + reserved goods
worksheet.write_formula('Q2', 'O2+P2',cell_format) # Total weight
worksheet.write_formula('Q3', 'O3+P3',cell_format) # Total CBM
worksheet.write_formula('Q4', 'O4+P4') # Total Cartoons
worksheet.write_formula('Q5', 'O5+P5',money_format) # Total SUM

In [ ]:
worksheet.freeze_panes(6, 0)

In [ ]:
worksheet.autofilter('A6:Q6')

In [ ]:
merge_format = workbook.add_format({
    # 'bold':     True,
    'align':    'center',
    'valign':   'vcenter',
    'font_name': 'Calibri',
    'font_size': 25 
})

In [ ]:
worksheet.merge_range('A1:L3', f'PRICE LIST {now}',merge_format)

In [ ]:
worksheet.set_zoom(67)

In [ ]:
writer.save()